In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [2]:
torch.cuda.is_available()

True

In [5]:
# Load data from solcast.csv
ghi_data = pd.read_csv('solcast.csv')

n_features = 1
sequence_length = 24

# Split data into train and validation sets
train_size = int(0.8 * len(input_sequences))
train_data = input_sequences[:train_size]
val_data = input_sequences[train_size:]

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, n_features)).reshape(-1, sequence_length, n_features)
X_val_scaled = scaler.transform(X_val.reshape(-1, n_features)).reshape(-1, sequence_length, n_features)
X_test_scaled = scaler.transform(X_test.reshape(-1, n_features)).reshape(-1, sequence_length, n_features)

# Create input sequences (e.g., using a sliding window)
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i : i + seq_length]
        sequences.append(seq)
    return np.array(sequences)

seq_length = 10  # Adjust as needed
input_sequences = create_sequences(ghi_data["normalized_ghi"].values, seq_length)


In [6]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Define model parameters
input_size = 1  # Number of features (GHI values)
hidden_size = 64
num_layers = 2
output_size = 1  # Predict one value (next GHI)
model = LSTMModel(input_size, hidden_size, num_layers, output_size)


In [7]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Split data into train and validation sets
train_size = int(0.8 * len(input_sequences))
train_data = input_sequences[:train_size]
val_data = input_sequences[train_size:]

# Train the model
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for seq in train_data:
        optimizer.zero_grad()
        seq = torch.tensor(seq, dtype=torch.float32)
        output = model(seq)
        loss = criterion(output, seq[-1])
        loss.backward()
        optimizer.step()

    # Validation loss
    model.eval()
    with torch.no_grad():
        val_loss = sum(criterion(model(torch.tensor(seq, dtype=torch.float32)), seq[-1]) for seq in val_data)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")


ValueError: LSTM: Expected input to be 2D or 3D, got 1D instead

---------------------**Activate API Endpoint** before proceeding-------------------------

In [ ]:
import requests


In [ ]:

# input_data = {
#     "input": [
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
        
#     ]
# }


In [ ]:

# response = requests.post('http://localhost:5000/predict', json=input_data).json()
# # print(response)

# threshold = 0.5
# RBP = response["output"]

# print("RBP =", RBP)
# if RBP >= threshold:
#     print("WARNING! Impending robbery.")


In [ ]:
for i in range(int(n_samples*0.15)):
    input_data = {
    "input":  X_test_scaled.tolist()[i] 
}
    response = requests.post('http://localhost:5000/predict', json=input_data).json()

    threshold = 0.5
    RBP = response["output"]

    print(i, "RBP =", RBP, "| actual RBP =", y_test.tolist()[i])
    if RBP >= threshold:
        print("WARNING! Impending robbery.")

In [ ]:
# study first doen lstm
# work on dynamic input
# try solar dataset